import the SimPy package 

In [1]:
#from simpy import initialize
#from SimPy.Simulation import *
import simpy
import random
import collections

_____

In [2]:
def distribucion_llegada(env,puestos,cont,n):
    while True:
    #for i in range(n):
        cont["NT"] += 1
        r = random.random()
        #print("Distribucion de generacion de tipo de cliente: %5.2f"%r)
        if r <= cont["PE"]:
            ce = Customer(name = "clienteE%02d"%cont["NT"])
            ia = random.uniform(cont["min"],cont["max"])
            yield env.timeout(ia)
            print("T=%5.2f %s: COLA EMERGENCIA - llegue con un IA: %5.2f"%(env.now,ce.name,ia))
            colas(puestos)
            env.process(ce.llamada_emergencia(env,puestos,cont))
        else: 
            cc = Customer(name = "clienteC%02d"%cont["NT"])
            ia = random.uniform(cont["min"],cont["max"])
            yield env.timeout(ia)
            print("T=%5.2f %s: COLA COMERCIAL - llegue con un IA: %5.2f"%(env.now,cc.name,ia))
            colas(puestos)
            env.process(cc.llamada_comercial(env,puestos,cont))

In [3]:
class Customer():
    """Evento llegadas/llamadas de los clientes"""
    def __init__(self,name):
        self.name = name
        
    def llamada_emergencia(self,env,puestos,cont):
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        emergencia, comercial = puestos[1].request(priority=0), puestos[0].request(priority=0)
        atendido = yield emergencia | comercial 
        #assert atendido == {emergencia}
        
        # Me atendieron por puestos comerciales?
        if comercial in atendido:
            cont["multitasking"]+=1
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta = generar_TA(cont)
            print("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto comercial despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto de emergencia"%(env.now,self.name))
            
        else:
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
            
            ta = generar_TA(cont)
            print("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto de emergencia"%(env.now,self.name))
            
            
    def llamada_comercial(self,env,puestos,cont):
        arrive = env.now
        with puestos[0].request(priority=1) as request:
            yield request
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta = generar_TA(cont)
            print("T=%5.2f, %s: COLA COMERCIAL - Me atendieron despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto comercial"%(env.now,self.name))
        

In [4]:
def colas(p):
    print ("Atendiendo:[c:%1d, e:%1d]"%(p[0].count,p[1].count))
    print ("En cola:[c:%1d, e:%1d]"%(len(p[0].queue),len(p[1].queue)))


In [5]:
def generar_TA(cont):
    #(a=5,b=615)
    a=5
    b=615
    ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
    
    while ta<a: 
        ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
        if ta>b:
            ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
            if ta>b:
                continue
            else:
                break
        
    while ta>b: 
        ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
        if ta<a:
            ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
            if ta<a:
                continue
            else:
                break
            
    return ta

Condiciones Iniciales

In [10]:
# Experimental data
NT = 0
maxTime = 14400 #segundos
maxClient = 10000
#random.seed(RANDOM_SEED)
PE = 0.12 #Porcentaje Emergencia
PC = 0.88 #Porcentaje Comercial
NEC = 0 #Cantidad de gente que espero en cola menor a 40 segundos
PEC = 0.0 #Porcentaje de espera en cola menor a 40 segundos

# TA - distribucion normal
mu = 267.64 
sigma = 297.40

# IA - uniforme
IAMin = 1
IAMax = 60

# Contadores
ci = {"multitasking":0,
     "mu":mu,
     "sigma":sigma,
     "min":IAMin,
     "max":IAMax,
     "PE":PE,
     "PC":PC,
     "emer":0,
     "com":0,
     "NEC":NEC,
     "NT":NT}

In [11]:
# Puestos de atencion - variables de control
print('Ingrese cantidad de puestos comerciales')
cantCom = int(input())
ci["com"] = cantCom
print('Ingrese cantidad de puestos de emergencia')
cantEmer = int(input())
ci["emer"] = cantEmer

Ingrese cantidad de puestos comerciales
60
Ingrese cantidad de puestos de emergencia
20


Modelo de Simulacion

In [12]:
env = simpy.Environment()
puestos = [simpy.PriorityResource(env,capacity=cantCom),
           simpy.PriorityResource(env,capacity=cantEmer)] # menor el numero priority, mayor es la prioridad
           
env.process(distribucion_llegada(env,puestos,ci,n=maxClient))
env.run(until=maxTime)

T=50.79 clienteC01: COLA COMERCIAL - llegue con un IA: 50.79
Atendiendo:[c:0, e:0]
En cola:[c:0, e:0]
T=50.79, clienteC01: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:1, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC01: 591.14
T=69.93 clienteC02: COLA COMERCIAL - llegue con un IA: 19.14
Atendiendo:[c:1, e:0]
En cola:[c:0, e:0]
T=69.93, clienteC02: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:2, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC02: 455.71
T=71.03 clienteC03: COLA COMERCIAL - llegue con un IA:  1.10
Atendiendo:[c:2, e:0]
En cola:[c:0, e:0]
T=71.03, clienteC03: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:3, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC03: 161.84
T=74.76 clienteC04: COLA COMERCIAL - llegue con un IA:  3.73
Atendiendo:[c:3, e:0]
En cola:[c:0, e:0]
T=74.76, clienteC04: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atend

Atendiendo:[c:20, e:12]
En cola:[c:0, e:0]
T=3736.66, clienteC122: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:21, e:12]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC122: 600.75
T=3738.85 clienteC123: COLA COMERCIAL - llegue con un IA:  2.19
Atendiendo:[c:21, e:12]
En cola:[c:0, e:0]
T=3738.85, clienteC123: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:22, e:12]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC123: 137.82
T=3765.84, clienteE121: Sali de un puesto de emergencia
T=3771.93, clienteC118: Sali de un puesto comercial
T=3791.98 clienteC124: COLA COMERCIAL - llegue con un IA: 53.12
Atendiendo:[c:21, e:12]
En cola:[c:0, e:0]
T=3791.98, clienteC124: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:22, e:12]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC124: 212.86
T=3817.21 clienteC125: COLA COMERCIAL - llegue con un IA: 25.23
Atendiendo:[c:22, e:12]
En cola:[c:0, e:0]


Atendiendo:[c:38, e:20]
En cola:[c:0, e:12]
Tiempo de Atencion generado para clienteC253: 108.39
T=7704.55, clienteC242: Sali de un puesto comercial
T=7714.44 clienteC254: COLA COMERCIAL - llegue con un IA: 17.98
Atendiendo:[c:37, e:20]
En cola:[c:0, e:12]
T=7714.44, clienteC254: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:38, e:20]
En cola:[c:0, e:12]
Tiempo de Atencion generado para clienteC254: 241.94
T=7736.57 clienteC255: COLA COMERCIAL - llegue con un IA: 22.13
Atendiendo:[c:38, e:20]
En cola:[c:0, e:12]
T=7736.57, clienteC255: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:39, e:20]
En cola:[c:0, e:12]
Tiempo de Atencion generado para clienteC255: 33.87
T=7769.99 clienteC256: COLA COMERCIAL - llegue con un IA: 33.42
Atendiendo:[c:39, e:20]
En cola:[c:0, e:12]
T=7769.99, clienteC256: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:40, e:20]
En cola:[c:0, e:12]
Tiempo de Atencion generado para clienteC256: 26.9

T=13751.43, clienteC432: Sali de un puesto comercial
T=13751.43, clienteE448: COLA EMERGENCIA - Me atendieron en un puesto comercial despues de esperar 405.05
Atendiendo:[c:60, e:20]
En cola:[c:22, e:39]
Tiempo de Atencion generado para clienteE448: 178.40
T=13758.16, clienteC433: Sali de un puesto comercial
T=13758.16, clienteE451: COLA EMERGENCIA - Me atendieron en un puesto comercial despues de esperar 370.43
Atendiendo:[c:60, e:20]
En cola:[c:21, e:39]
Tiempo de Atencion generado para clienteE451: 27.80
T=13764.35 clienteC460: COLA COMERCIAL - llegue con un IA: 30.52
Atendiendo:[c:60, e:20]
En cola:[c:21, e:39]
T=13785.96, clienteE451: Sali de un puesto de emergencia
T=13807.41, clienteC435: Sali de un puesto comercial
T=13807.41, clienteC436: COLA COMERCIAL - Me atendieron despues de esperar 718.47
Atendiendo:[c:60, e:20]
En cola:[c:21, e:39]
Tiempo de Atencion generado para clienteC436: 129.73
T=13807.86 clienteC461: COLA COMERCIAL - llegue con un IA: 43.51
Atendiendo:[c:60, e:20

Results

In [13]:
print("===========================================================================")
print("RESULTADOS:")
print("Clientes totales: %1d"%ci["NT"])
PEC = (ci["NEC"] / ci["NT"]) * 100
# Porcentaje de espera menor a 40 segundos
print("Porcentaje de espera en cola menor a 40 segundos: %5.2f"%PEC)
# Objetio: Mayor o igual al 90%

print('Cantidad de puestos comerciales',cantCom)
print('Cantidad de puestos de emergencia',cantEmer)
#print("Cantidad de personales multitasking",ci["multitasking"])

RESULTADOS:
Clientes totales: 484
Porcentaje de espera en cola menor a 40 segundos: 85.95
Cantidad de puestos comerciales 60
Cantidad de puestos de emergencia 20
